### Importing the File
1. import dask
2. export from VAN as txt, and save to this directory
3. read table 



In [40]:
### if first time installing dask:

#!python -m pip install "dask[complete]" 

import dask.dataframe as dd

In [2]:
#### if not Fryda, make sure to change the working directory to where you are saving everything in your computer
### change the name to the folder where you save the files and also where you will launch jupyter notebook

# os.chdir('../')


In [3]:
###reading a txt file

####add pathname to your file below

CHfile = "/Users/frydafly/Downloads/2020CH-14654181676.txt"

In [12]:
#### load file into dataframe
#### had to add engine = "python" to fix a unicode error

df = dd.read_table(CHfile, engine = "python", encoding='cp1252')

### for reading a csv file, see below
# ch = pd.read_csv("")

In [41]:
#### take a look at all the columns
df.head(2)

,Voter File VANID,DWID,VoterName,ResultShortName,CreatedBy,CanvassedBy,CommitteeName,DateCreated,DateCanvassed,ContactTypeName,Campaign,Content,InputTypeName
0,24050775,984972613.0,"Gomez, Luz D",Left Message,Hustle STV API,"API, Hustle STV",HispanicFed,06/09/2020,06/09/2020,SMS Text,2020 Census,NaN,API
1,24050807,984972645.0,"Piazzolla, Jamilla Rosa",Texted,Hustle STV API,"API, Hustle STV",NYIC,09/21/2020,09/21/2020,SMS Text,2020 Census,NaN,API


In [19]:
## count number of rows

df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 13 entries, Voter File VANID to InputTypeName 
dtypes: object(10), float64(2), int64(1)

## Clean Dataframe
1. Drop extra columns
2. Reorder Columns 
3. Convert DWID from float to integer
4. uppercase Result, ContactType, InputType
5. Remove NaN, and make blank
6. Standardize committee names based on translation table?

In [20]:
# drop VoterName and Content Columns

df1 = df.drop(columns=['VoterName', 'Content'])

#look at the new columns
df1.head(2)

,Voter File VANID,DWID,ResultShortName,CreatedBy,CanvassedBy,CommitteeName,DateCreated,DateCanvassed,ContactTypeName,Campaign,InputTypeName
0,24050775,984972613.0,Left Message,Hustle STV API,"API, Hustle STV",HispanicFed,06/09/2020,06/09/2020,SMS Text,2020 Census,API
1,24050807,984972645.0,Texted,Hustle STV API,"API, Hustle STV",NYIC,09/21/2020,09/21/2020,SMS Text,2020 Census,API


In [42]:
#### reorder the columns -- not working right now because of dask!

df2 = df1[['Voter File VANID', 'DWID', 'ResultShortName', 'DateCreated', 'DateCanvassed', 'ContactTypeName', 'InputTypeName ', 'CommitteeName', 'CreatedBy', 'CanvassedBy', 'Campaign']]


#look at new columns
df2.head(2)

,Voter File VANID,DWID,ResultShortName,DateCreated,DateCanvassed,ContactTypeName,InputTypeName,CommitteeName,CreatedBy,CanvassedBy,Campaign
0,24050775,984972613.0,Left Message,06/09/2020,06/09/2020,SMS Text,API,HispanicFed,Hustle STV API,"API, Hustle STV",2020 Census
1,24050807,984972645.0,Texted,09/21/2020,09/21/2020,SMS Text,API,NYIC,Hustle STV API,"API, Hustle STV",2020 Census


In [43]:
# Remove NaN

df2 = df1.fillna('')

In [44]:
# uppercase for Result, ContactType, InputType

df2['ResultShortName'] = df2['ResultShortName'].str.upper() 
df2['ContactTypeName'] = df2['ContactTypeName'].str.upper() 
df2['InputTypeName '] = df2['InputTypeName '].str.upper() 

#display results
df2.head(2)

,Voter File VANID,DWID,ResultShortName,CreatedBy,CanvassedBy,CommitteeName,DateCreated,DateCanvassed,ContactTypeName,Campaign,InputTypeName
0,24050775,9.84973e+08,LEFT MESSAGE,Hustle STV API,"API, Hustle STV",HispanicFed,06/09/2020,06/09/2020,SMS TEXT,2020 Census,API
1,24050807,9.84973e+08,TEXTED,Hustle STV API,"API, Hustle STV",NYIC,09/21/2020,09/21/2020,SMS TEXT,2020 Census,API


In [45]:
# check types to make sure it makes sense

df2.dtypes

Voter File VANID      int64
DWID                float64
ResultShortName      object
CreatedBy            object
CanvassedBy          object
CommitteeName        object
DateCreated          object
DateCanvassed        object
ContactTypeName      object
Campaign             object
InputTypeName        object
dtype: object

In [77]:
df3['Voter File VANID'] = df2['Voter File VANID'].fillna(0)

In [46]:
##convert DWID to int

#fill na by 0, since that caused an issue the first time
df2['DWID'] = df2['DWID'].fillna(0)

#convert DWID to int
df2['DWID'] = df2['DWID'].astype('int64') 

##convert inputtypename & campaign to object
df2 = df2.astype({"InputTypeName ":'object', "Campaign":'object'})  

#display types again
df2.dtypes

Voter File VANID     int64
DWID                 int64
ResultShortName     object
CreatedBy           object
CanvassedBy         object
CommitteeName       object
DateCreated         object
DateCanvassed       object
ContactTypeName     object
Campaign            object
InputTypeName       object
dtype: object

## Clean Out Org Names -- not ready in Dask
1. Remove orgs that are not NYCET Partner Orgs
2. Rename org names to fit names in server

In [60]:
# List distinct CommitteeNames

#orgs = df2.drop_duplicates(subset=['CommitteeName']).compute()

#print(orgs)

In [72]:
## Remove orgs that are not NYCET partner org

#list out index for rows that are not partners
df3 = df2[(df.CommitteeName != 'Planned Parenthood c3')]

#drop rows with these indexes
#df2.drop(indexNames, axis=1)

#check the distinct committeenames again
#df2.CommitteeName.unique()

In [62]:
# rename existing names to fit orgnames in server

#df2['CommitteeName'] = df['CommitteeName'].replace(['Minkwon','Make the Road'],['MinKwon', 'MRNY'])

#display new names
#df2.CommitteeName.unique()

## Narrow to Proper Date Range

In [37]:
## this isn't ready yet, and we didn't need it yet, so I'll skip

## Check our work

In [74]:
# count number of rows

df3.count()

Dask Series Structure:
npartitions=1
Campaign            int64
Voter File VANID      ...
dtype: int64
Dask Name: dataframe-count-agg, 376 tasks

## Print New CSV

In [ ]:
df3.to_csv(r'/Users/frydafly/Downloads/serverupdate/CH.csv', index = False, encoding='utf-8' )

## If needed, split the file into manageable chunks